<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 01
Exercise 01 of the course 01ZLMA. 

## Contents


*   Recap of multivariable linear regression (FJFI course 01RAD)
*   Discussion about organizational/run of the course
*   Introdaction into Google colab notebooks (R version https://colab.to/r or Python versions)


To load required libraries (with R runtime type):


In [ ]:
#install.packages("car")
#install.packages("reactable")
#install.packages("GGally")
#install.packages("pander")
#library(car)
library(tidyverse)
library(MASS)
library(knitr)

#library(pander)

library(GGally)
library(reactable)
library(gridExtra)


#For sure: set dplyr functions
select    <- dplyr::select;
rename    <- dplyr::rename;
mutate    <- dplyr::mutate; 
summarize <- dplyr::summarize;
arrange   <- dplyr::arrange;
slice     <- dplyr::slice;
filter    <- dplyr::filter;
recode    <- dplyr::recode


Your workong directory is in the cloud. You can mount your Google drive, or your local hard drive
(https://stackoverflow.com/questions/56679549/how-to-mount-google-drive-to-r-notebook-in-colab)
 (https://github.com/katewall/medium_tutorials/blob/main/210630_Medium_ColabwithR.ipynb). 

In [ ]:
R.version.string
getwd()

## Regression analysis


### Fish Catch Dataset

159 fishes of 7 species are caught and measured. Altogether there are
8 variables.  All the fishes are caught from the same lake
(Laengelmavesi) near Tampere in Finland.

SOURCES:
Brofeldt, Pekka: Bidrag till kaennedom on fiskbestondet i vaera
        sjoear. Laengelmaevesi. T.H.Jaervi: Finlands Fiskeriet  Band 4,
        Meddelanden utgivna av fiskerifoereningen i Finland.
        Helsingfors 1917

VARIABLE DESCRIPTIONS:

1.  Obs       Observation number ranges from 1 to 159
2.  Species   (Numeric)
        Code Finnish  Swedish    English        Latin      
         1   Lahna    Braxen     Bream          Abramis brama
         2   Siika    Iiden      Whitewish      Leusiscus idus
         3   Saerki   Moerten    Roach          Leuciscus rutilus
         4   Parkki   Bjoerknan  Blicca         Abramis bjrkna
         5   Norssi   Norssen    Smelt          Osmerus eperlanus
         6   Hauki    Jaedda     Pike           Esox lucius
         7   Ahven    Abborre    Perch          Perca fluviatilis

3.  Weight      Weight of the fish (in grams)
4.  Length1     Length from the nose to the beginning of the tail (in cm)
5.  Length2     Length from the nose to the notch of the tail (in cm)
6.  Length3     Length from the nose to the end of the tail (in cm)
7.  Height%     Maximal height as % of Length3
8.  Width%      Maximal width as % of Length3
9.  Sex         1 = male 0 = female



In [ ]:
#@title Read dataset (without colnames)
fishcatch <- read.table("http://jse.amstat.org/datasets/fishcatch.dat.txt") %>%
  rename(Obs =V1,
         Species = V2,
         Weight = V3,
         Len1 = V4,
         Len2 = V5,
         Len3 = V6,
         Height = V7,
         Width = V8,
         Sex = V9
         )

In [ ]:
head(fishcatch)
fishcatch %>% 
   summary() %>%
   kable(format = "pipe")
    


|   |     Obs      |   Species    |    Weight     |     Len1     |     Len2     |     Len3     |    Height    |    Width     |     Sex       |
|:--|:-------------|:-------------|:--------------|:-------------|:-------------|:-------------|:-------------|:-------------|:--------------|
|   |Min.   :  1.0 |Min.   :1.000 |Min.   :   0.0 |Min.   : 7.50 |Min.   : 8.40 |Min.   : 8.80 |Min.   :14.50 |Min.   : 8.70 |Min.   :0.0000 |
|   |1st Qu.: 40.5 |1st Qu.:2.000 |1st Qu.: 120.0 |1st Qu.:19.05 |1st Qu.:21.00 |1st Qu.:23.15 |1st Qu.:24.25 |1st Qu.:13.40 |1st Qu.:0.0000 |
|   |Median : 80.0 |Median :5.000 |Median : 272.5 |Median :25.20 |Median :27.30 |Median :29.40 |Median :27.10 |Median :14.60 |Median :0.0000 |
|   |Mean   : 80.0 |Mean   :4.497 |Mean   : 398.7 |Mean   :26.25 |Mean   :28.42 |Mean   :31.23 |Mean   :28.31 |Mean   :14.12 |Mean   :0.2361 |
|   |3rd Qu.:119.5 |3rd Qu.:7.000 |3rd Qu.: 650.0 |3rd Qu.:32.70 |3rd Qu.:35.50 |3rd Qu.:39.65 |3rd Qu.:37.60 |3rd Qu.:15.30 |3rd Qu.:0.0000 |
|   |Max.   :159.0 |Max.   :7.000 |Max.   :1650.0 |Max.   :59.00 |Max.   :63.40 |Max.   :68.00 |Max.   :44.50 |Max.   :20.90 |Max.   :1.0000 |
|   |NA            |NA            |NA's   :1      |NA            |NA            |NA            |NA            |NA            |NA's   :87     |

In [ ]:
#reactable(fishcatch)

In [ ]:
mutate_cond <- function(.data, condition, ..., envir = parent.frame()) {
  condition <- eval(substitute(condition), .data, envir)
  .data[condition, ] <- .data[condition, ] %>% mutate(...)
  .data
}

In [ ]:
fish <- fishcatch %>%
  drop_na(Weight) %>% 
  mutate_cond(is.na(Sex), Sex = "unknown") %>%
  mutate(Sex = as.factor(Sex)) %>%
  mutate(Sex = recode(Sex,"1" = "male", "0" = "female")) %>%
  mutate(Species = factor(Species)) %>%
  select(-Obs)
fish %>% summary() %>% kable()


In [ ]:
options(jupyter.plot_scale=0.8)
options(warn=-1)

fish %>% ggpairs(., legend = 1, columns = 2:8,
                   aes(color = Species),
                   upper = list(continuous = wrap("cor", size=2)),
                   lower = list(continuous = wrap("points", alpha = 0.3,    size=0.4)))+                 
  theme(legend.position = "bottom") + 
  labs(color = "Engine Type")

In [ ]:
plot1 <- ggplot(fish %>% mutate(Species = as.factor(Species)),
                         aes(x=Species, y=Weight, fill=Sex)) +
  geom_boxplot(size = 1, notch = F) +
  geom_jitter(aes(Species,Weight),
              position=position_jitter(width=0.1,height=0),
              alpha=0.5,
              size=0.8,
              show.legend=F)+
  labs(title="Weight vs Species", y="Weight", x="Species") +
  theme_bw() +  
  labs(title="Weight vs Species",
       subtitle="Distinguish between Sex",
       caption="version: 001") 
plot1

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3)

ggplot(fish, aes(x=Sex, y=Weight)) +
  geom_boxplot(size = 1, notch = F) +
  geom_jitter(aes(Sex,Weight),
              position=position_jitter(width=0.1,height=0),
              alpha=0.5,
              size=0.8,
              show.legend=F)+
  theme_bw()  


# Your turn 01

* Recode Species, instead numeric use fish names.
* Which variables look promising, which variables can be omitted.
* Edit the dataset
* Plot Height vs. Weight, Leng3 vs. Weight, Width vs. Weight

# Regression


...

In [ ]:
summary(fish)

In [ ]:
Fish <- fish %>%
  filter(Species %in% c(1,3,7)) %>%
  mutate(Species = recode(Species,"1" = "Bream",
                                  "2" = "Whitewish",
                                  "3" = "Roach",
                                  "4" = "Blicca",
                                  "5" = "Smelt",
                                  "6" = "Pike",
                                  "7" = "Perch")) %>%
  mutate(Species = fct_drop(Species))%>%                            
  select(-Sex) %>%
  filter(Weight != 0)
head(Fish)
summary(Fish)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5, repr.plot.res = 90)

Fish <- Fish %>% mutate(Height = Height*Len3,
                        Width = Width*Len3)

Height_Weight <- ggplot(Fish,aes(x=Height,y=Weight,col=Species))+
                 geom_point(size=5,alpha=0.6)+
                 geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,formula="y~x")

LengthV_Weight <- ggplot(Fish,aes(x=Len3,y=Weight,col=Species))+
                  geom_point(size=5,alpha=0.6)+
                  geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,formula="y~x")

Width_Weight <- ggplot(Fish,aes(x=Width,y=Weight,col=Species))+
                 geom_point(size=5,alpha=0.6)+
                 geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,formula="y~x")

#Height_Weight
#LengthV_Weight
#Width_Weight

grid.arrange(Height_Weight, LengthV_Weight, Width_Weight, ncol = 3)

In [ ]:
Height_Weight_log <- ggplot(Fish,aes(x=log(Width),y=log(Weight),col=Species))+
                 geom_point(size=5,alpha=0.6)+
                 geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,,formula="y~x")

LengthV_Weight_log <- ggplot(Fish,aes(x=log(Len3),y=log(Weight),col=Species))+
                  geom_point(size=5,alpha=0.6)+
                  geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,,formula="y~x")

Width_Weight_log  <- ggplot(Fish,aes(x=log(Height),y=log(Weight),col=Species))+
                 geom_point(size=5,alpha=0.6)+
                 geom_smooth(col="grey40",method = "lm",se=F,lty="dashed",lwd=2,,formula="y~x")


grid.arrange(Height_Weight_log, LengthV_Weight_log, Width_Weight_log, ncol = 3)


In [ ]:
# Model with all variables and interaction till 2rd order ... 
m0  <- lm(Weight ~ (.)^2, data = Fish)
summary(m0)
# Ufff 

Q: 
* How can you interpret previous result?
* Comment and discussion: how to select model.


In [ ]:
m0_BIC  <- stepAIC(m0, k=log(dim(Fish)[1]))


m0_AIC  <- stepAIC(m0)
summary(m0_BIC)
summary(m0_AIC)

In [ ]:
install.packages("leaps")
library(leaps)

In [ ]:
best_subset <- regsubsets(Weight ~ (.)^2, Fish, nvmax = 20,really.big=T)
results <- summary(best_subset)
plot(best_subset)


In [ ]:
#  plot results
# source: https://afit-r.github.io/model_selection
tibble(predictors = 1:20,
       adj_R2 = results$adjr2,
       Cp = results$cp,
       BIC = results$bic) %>%
  gather(statistic, value, -predictors) %>%
  ggplot(aes(predictors, value, color = statistic)) +
  geom_line(show.legend = F) +
  geom_point(show.legend = F) +
  facet_wrap(~ statistic, scales = "free")

# Your turn 02

* Is there problem with multicolinearity? If so, how can we cope with it.
* Try to find such a good model (based on adjuste R^2) with a maximum of 7 covariates. 

Hint: try to use expert knowledge.


In [ ]:
m1 <- lm(Weight ~ (.), data = Fish)
summary(m1)

In [ ]:
# VIF
1/(1-(summary(lm(Len1 ~.,data = Fish %>% select(-Weight)))$r.squared))
1/(1-(summary(lm(Len2 ~.,data = Fish %>% select(-Weight)))$r.squared))
1/(1-(summary(lm(Len3 ~.,data = Fish %>% select(-Weight)))$r.squared))
1/(1-(summary(lm(Height ~.,data = Fish %>% select(-Weight)))$r.squared))
1/(1-(summary(lm(Width  ~.,data = Fish %>% select(-Weight)))$r.squared))

In [ ]:
kappa(scale(data.matrix(Fish)))
kappa(scale(data.matrix(Fish %>% select(-Len1,-Len2))))

In [ ]:
m1 <- lm(Weight ~ Species+Len2:Len3:Height+Len2:Len3:Width, data = Fish)
summary(m1)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 90)

par(mfrow = c(2, 2))
plot(m1, pch = 20, col = "blue4", lwd = 2)


In [ ]:
log_m1 <- lm(log(Weight) ~ Species+Len2:Len3:Height+Len2:Len3:Width, data = Fish)
summary(log_m1)

In [ ]:
par(mfrow = c(2, 2))
plot(log_m1, pch = 20, col = "blue4", lwd = 2)


In [ ]:
log_m1 <- lm(log(Weight) ~ Species+Len2:Len3:Height+Len2:Len3:Width, data = Fish[-54,])
par(mfrow = c(2, 2))
plot(log_m1, pch = 20, col = "blue4", lwd = 2)


In [ ]:
# Box-Cox transformation
BC_m0  <- lm(Weight ~ Species+I(Len3^2) +Len3:Height:Width, data = Fish)
summary(BC_m0)
bc     <- boxcox(m1, lambda = seq(-1,1 , 1/100))
lambda <- bc$x[which.max(bc$y)]
lambda

In [ ]:
BC_m1 <- lm(Weight^(1/2) ~ Species+I(Len3^2) + Len3:Height+Len3:Width, data = Fish)
summary(BC_m1)

#Fish$Weight_lambda = (Fish$Weight^lambda-1)/lambda
#BC_m1 <- lm(Weight_lambda  ~ Species+I(Len3^2) + Len3:Height+Len3:Width, data = Fish)

summary(BC_m1)
par(mfrow = c(2, 2))
plot(BC_m1, pch = 20, col = "blue4", lwd = 2)


In [ ]:
#install.packages("pbkrtest")
#install.packages("lme4")
#install.packages("RcppEigen")
#install.packages("car")
#library(car)

In [ ]:
m_f <- lm(log(Weight) ~ Species+log(Len3)*log(Height)*log(Width), data = Fish[-54,])
summary(m_f)

#Fish$Weight_lambda = (Fish$Weight^lambda-1)/lambda
#BC_m1 <- lm(Weight_lambda  ~ Species+I(Len3^2) + Len3:Height+Len3:Width, data = Fish)

summary(m_f)
par(mfrow = c(2, 2))
plot(m_f, pch = 20, col = "blue4", lwd = 2)
